In [129]:
#importing Libraries
import numpy as np
import pandas as pd
import pprint
eps = np.finfo(float).eps
#reading the data from the csv file
data=pd.read_csv("lab01_dataset_1.csv")
data.head()

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,Medium,91,No
2,Happy,High,52,No
3,Neutral,Low,83,No
4,Neutral,Medium,48,No


In [130]:
print(data['Score_Info'].unique())
print(data['Effort'].unique())
print(data['Mood'].unique())

KeyError: 'Score_Info'

In [108]:
# Entropy of the target attribute values
def find_entropy(df):
    target = df.keys()[-1]  # The last dataframe column is the target attribute (playGolf)
    entropy = 0
    values = df[target].unique()
    # for each value in the target playGolf attribute values
    for value in values:
        # ratio of values occurring and entropy
        fraction = df[target].value_counts()[value] / len(df[target])
        entropy += -fraction * np.log2(fraction)
    return entropy

In [109]:
def find_entropy_attribute(df, attribute):
    target = df.keys()[-1]
    target_variables = df[target].unique()  
    # unique values in target playGolf attribute (Yes, No)
    variables = df[attribute].unique() # Identify Sunny, Overcast, Rainy
    # attribute entropy
    # Variables=[sunny, sunny....5, overcast1.....overcast4, Rainy1...Ra5 ]
    entropy2 = 0
    # for each attribute value in attribute values
    for variable in variables:
        
        # value entropy
        entropy = 0
        # for each target value in target values (yes/no)
        for target_variable in target_variables:
            
   # frequency of attribute and target values (boolean indexing, pandas dataframe filtering)
          num = len(df[attribute][df[attribute] == variable][df[target] == target_variable])
          den = len(df[attribute][df[attribute] == variable])
          fraction = num / (den + eps)
          entropy += -fraction * np.log2(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)

In [110]:
def bestClassifier(df):
    # Entropy_att = []
    # information gain array for all attributes
    IG = []
    # for all attributes excluding target
    for key in df.keys()[:-1]:
        # Entropy_att.append(find_entropy_attribute(df,key))
        # calculate and record information gain value
        IG.append(find_entropy(df) - find_entropy_attribute(df, key)) 
    return df.keys()[:-1][np.argmax(IG)]  

In [111]:
# function for creating subtable
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)
def ID3split(df, tree=None):
    target = df.keys()[-1]
    
    # Here we build our decision tree
    # Get attribute with maximum information gain
    node = bestClassifier(df) # 0.247
    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attributeValues = np.unique(df[node])
    # Create an empty dictionary to create tree (recursive-friendly definition)
    if tree is None:               #  Outlook ->root node attribute
        tree = {}
        tree[node] = {}
    # following loop recursively calls ID3split to create and add to the tree
    # it runs till the tree is pure (leaf (result) node branches are added to the tree)
    for value in attributeValues:
        
        # get the subtable from current node based on the value
        subtable = get_subtable(df, node, value)
        # get the most common target value in the subtable
        targetValues, counts = np.unique(subtable[target], return_counts=True)
        
        # if the subtable is empty, assign the leaf node to the most common target value
        if len(counts) == 1:
            tree[node][value] = targetValues[0]
        
        else:
            # recursively call ID3 to create subtrees
            tree[node][value] = ID3split(subtable)  # Calling the function recursively
    return tree

In [112]:
decisionTree = ID3split(data)
print(decisionTree)
pprint.pprint(decisionTree)

{'Score_Info': {'High_Score': 'Yes', 'Low_Score': 'Yes', 'Mild_Score': 'No', 'Ultra_Score': 'No'}}
{'Score_Info': {'High_Score': 'Yes',
                'Low_Score': 'Yes',
                'Mild_Score': 'No',
                'Ultra_Score': 'No'}}


In [113]:
data.head()

,Mood,Effort,Score_Info,Output
0,Happy,Low,Low_Score,Yes
1,Happy,High,Low_Score,Yes
2,Sad,Low,Low_Score,Yes
3,Happy,Low,Low_Score,Yes
4,Neutral,Medium,Mild_Score,No


## ------------------------------------------------------------------------------------------------------------------------------------------------------------

In [189]:
import pandas as pd
import numpy as np
import pprint
data = pd.read_csv('lab01_dataset_1.csv')
data

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,Medium,91,No
2,Happy,High,52,No
3,Neutral,Low,83,No
4,Neutral,Medium,48,No
5,Neutral,High,61,No
6,Sad,Low,40,Yes
7,Sad,Medium,98,No
8,Sad,High,73,Yes
9,Happy,Low,44,Yes


In [190]:
data = data.sort_values('Score', ignore_index=True)
data.reindex(np.arange(len(data['Score'].index)))

# data['Score'] = pd.cut(data['Score'], bins=4, labels=[1, 2, 3, 4])
data

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,High,39,Yes
2,Sad,Low,40,Yes
3,Happy,Low,44,Yes
4,Neutral,Medium,48,No
5,Neutral,High,50,No
6,Happy,High,52,No
7,Neutral,High,61,No
8,Happy,Low,66,No
9,Sad,Low,69,No


In [191]:
def splitPoint(data):
    attributeDataFrame = pd.DataFrame({ 'Score' : data['Score'], 'Output' : data['Output']})
    attributeDataFrame.reindex(np.arange(len(attributeDataFrame['Score'].index)))

    outputvalue = attributeDataFrame['Output'][0]

    splitPointDict = {}

    for i in range(1, len(attributeDataFrame['Score'])):
        if(attributeDataFrame['Output'][i] != outputvalue):
            mean = (attributeDataFrame['Score'][i-1] + attributeDataFrame['Score'][i])/2
            splitPointDict[i-1] = mean
            outputvalue = attributeDataFrame['Output'][i]
        
    pprint.pprint(splitPointDict)
    return attributeDataFrame, splitPointDict
attributeDataFrame, splitPoint = splitPoint(data)

for i in splitPoint:
    split = splitPoint[i]
    print(splitPoint[i])
    ltYes = 0
    ltNo = 0
    gtYes = 0
    gtNo = 0

    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'Yes':
            ltYes+=1
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'No':
            ltNo+=1

        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'Yes':
            gtYes+=1
        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'No':
            gtNo+=1

    print(ltYes , " | " , ltNo  , " | " , gtYes , " | " , gtNo)


columnsList = {}

for i in splitPoint:
    split = splitPoint[i]
    # "Score_" + str(split)
    scoreList=[]
    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split:
            scoreList.append('T') # ("Score_" + str(split) + "True")
        else: 
            scoreList.append('F') # ("Score_" + str(split) + "False")

    columnsList["Score_" + str(split)] = scoreList

pprint.pprint(columnsList)

data['Score_81.5'] = columnsList['Score_81.5']
data['Score_46.0'] = columnsList['Score_46.0']
data['Score_69.5'] = columnsList['Score_69.5']

# feature Score is dropped
data = pd.DataFrame({
    'Mood' : data['Mood'], 
    'Effort' : data['Effort'], 
    'Score_46.0' : data['Score_46.0'], 
    'Score_69.5' : data['Score_69.5'], 
    'Score_81.5' : data['Score_81.5'],
    'Output' : data['Output']
})

data.head()

{3: 46.0, 9: 69.5, 13: 81.5}
46.0
4  |  0  |  4  |  11
69.5
4  |  6  |  4  |  5
81.5
8  |  6  |  0  |  5
{'Score_46.0': ['T',
                'T',
                'T',
                'T',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F'],
 'Score_69.5': ['T',
                'T',
                'T',
                'T',
                'T',
                'T',
                'T',
                'T',
                'T',
                'T',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F',
                'F'],
 'Score_81.5': ['T',
                'T',
                'T',
                'T',
           

,Mood,Effort,Score_46.0,Score_69.5,Score_81.5,Output
0,Happy,Low,T,T,T,Yes
1,Happy,High,T,T,T,Yes
2,Sad,Low,T,T,T,Yes
3,Happy,Low,T,T,T,Yes
4,Neutral,Medium,F,T,T,No


In [192]:
def entropy(cols):
    counts = cols.value_counts()
    probs = counts/len(cols)
    return -np.sum(probs*np.log2(probs))

def info_gain(data, split_attr, target_attr):

    total_entropy = entropy(data[target_attr])
    
    values = data[split_attr].unique()
    weighted_entropy = 0
    
    for value in values:
        subset = data[data[split_attr]==value]
        weighted_entropy += (len(subset)/len(data)) * entropy(subset[target_attr])
        
    return total_entropy - weighted_entropy

In [193]:
def id3(data, original_data, features, target_attribute):

    if len(data[target_attribute].unique()) == 1:
        return data[target_attribute].unique()[0]

    elif data.empty:
        return original_data[target_attribute].value_counts().idxmax()

    elif len(features) == 0:
        return data[target_attribute].value_counts().idxmax()

    else:
        gains = [info_gain(data, feature, target_attribute) for feature in features]
        best_feature = features[np.argmax(gains)]
        
        tree = {best_feature: {}}
        remaining_features = [f for f in features if f != best_feature]
        
        for value in data[best_feature].unique():
            sub_data = data[data[best_feature] == value]
            subtree = id3(sub_data, original_data, remaining_features, target_attribute)
            tree[best_feature][value] = subtree
            
        return tree
        
tree = id3(data, data, data.columns[:-1], 'Output')
print(tree)

pprint.pprint(tree)

{'Score_46.0': {'T': 'Yes', 'F': {'Score_69.5': {'T': 'No', 'F': {'Score_81.5': {'T': 'Yes', 'F': 'No'}}}}}}
{'Score_46.0': {'F': {'Score_69.5': {'F': {'Score_81.5': {'F': 'No',
                                                          'T': 'Yes'}},
                                     'T': 'No'}},
                'T': 'Yes'}}
